In [1]:
# Expecting at this point we already have 
# 1. Feat.npy generated, 2. corresponding csv files 3. train/test races .txt file
import os
import glob
import cv2
import copy, time

import torch
import numpy as np

from util.train_helper import *
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# Set up Parameters
# HVT small batch for testing
train_races_txt = "G:/tim/Project/Connectivity_train_test/HVT_small_train_races.txt"
test_races_txt = "G:/tim/Project/Connectivity_train_test/HVT_small_test_races.txt"
trip_feat_train_dir = "D:/Bitbucket/triplettraining/feat/HVT_train"
trip_feat_test_dir = "D:/Bitbucket/triplettraining/feat/HVT_test"
cap_gt_dir = "G:/tim/Project/Connectivity_train_test/real_data/HVT-2018-2019_full_recall"

# HVT Full
# train_races_txt = "G:/tim/Project/Connectivity_train_test/HVT_train_races.txt"
# test_races_txt = "G:/tim/Project/Connectivity_train_test/HVT_test_races.txt"
# trip_feat_train_dir = "D:/Bitbucket/triplettraining/feat/HVT_train"
# trip_feat_test_dir = "D:/Bitbucket/triplettraining/feat/HVT_test"
# cap_gt_dir = "G:/tim/Project/Connectivity_train_test/real_data/HVT-2018-2019_full_recall"

# Kranji Full
# train_races_txt = "G:/tim/Project/Connectivity_train_test/Kranji_train_races.txt"
# test_races_txt = "G:/tim/Project/Connectivity_train_test/Kranji_test_races.txt"
# trip_feat_train_dir = "D:/Bitbucket/triplettraining/feat/Kranji_train"
# trip_feat_test_dir = "D:/Bitbucket/triplettraining/feat/Kranji_test"
# cap_gt_dir = "G:/tim/Project/Connectivity_train_test/real_data/Kranji-2018_verified_26races"

param_dict = get_default_param()

batch_size = 256
track = "Kranji"
time_str = param_dict["time_str"]
proj_name = f"Connectivity_batch_{batch_size}_{track}_{time_str}"
param_dict["project_name"] = proj_name
param_dict["train_batch_size"] = batch_size
param_dict["test_batch_size"] = batch_size

In [3]:
# Construct the dataset from the triplets files
from datasets.dataset import ConnDataset
from torch.utils.data import Dataset, TensorDataset, DataLoader, WeightedRandomSampler
conn_train_dataset = ConnDataset(train_races_txt, trip_feat_train_dir, cap_gt_dir)
conn_test_dataset = ConnDataset(test_races_txt, trip_feat_test_dir, cap_gt_dir)

# This part is to balance the data, 
# to ensure even distribution of the data within batch
# If WeightedRandomSampler is being used, maximum 2**24 data is limited,
# Breaking down the dataset into smaller part?
labels = conn_train_dataset.label_classes
label_unique, counts = np.unique(labels, return_counts = True)
class_weights = [sum(counts) / c for c in counts]
example_weights = [class_weights[e] for e in labels]
train_sampler = WeightedRandomSampler(example_weights, len(labels))

labels = conn_test_dataset.label_classes
label_unique, counts = np.unique(labels, return_counts = True)
class_weights = [sum(counts) / c for c in counts]
example_weights = [class_weights[e] for e in labels]
test_sampler = WeightedRandomSampler(example_weights, len(labels))

train_dataloader = DataLoader(conn_train_dataset, sampler = train_sampler, batch_size = param_dict['train_batch_size'])
test_dataloader = DataLoader(conn_test_dataset, sampler = test_sampler, batch_size = param_dict['val_batch_size'])

Total 12 races to be processed.
Processing HVT-20180905-01 (1 / 12)...
Processing HVT-20180912-03 (2 / 12)...
Processing HVT-20181018-05 (3 / 12)...
Processing HVT-20181028-06 (4 / 12)...
Processing HVT-20181121-01 (5 / 12)...
Processing HVT-20181128-05 (6 / 12)...
Processing HVT-20181212-08 (7 / 12)...
Processing HVT-20190116-04 (8 / 12)...
Processing HVT-20190213-03 (9 / 12)...
Processing HVT-20190227-08 (10 / 12)...
Processing HVT-20190320-05 (11 / 12)...
Processing HVT-20190327-03 (12 / 12)...
Total 5 races to be processed.
Processing HVT-20180905-02 (1 / 5)...
Processing HVT-20180926-02 (2 / 5)...
Processing HVT-20181003-08 (3 / 5)...
Processing HVT-20181010-01 (4 / 5)...
Processing HVT-20181010-08 (5 / 5)...


In [6]:
from model.conn import ConnNet

conn_net = ConnNet().to(param_dict['device'])

dataloaders = {"train":train_dataloader, "val":test_dataloader}

loss = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(conn_net.parameters(), lr = 0.001, momentum = 0.9)

# Train Model
conn_net = train_model(conn_net, dataloaders, loss, optimizer, param_dict = param_dict)

Start training ...


Epoch 0/99
----------


val Loss: 0.6899 Acc: 0.5002
train Loss: 0.0666 Acc: 0.9799
saving trained model


Epoch 1/99
----------


train Loss: 0.0576 Acc: 0.9807


...

train Loss: 0.0133 Acc: 0.9954


Epoch 98/99
----------


train Loss: 0.0131 Acc: 0.9955


Epoch 99/99
----------


val Loss: 0.3197 Acc: 0.9944
train Loss: 0.0132 Acc: 0.9954


In [10]:
len(conn_test_dataset.inputs)

2560399